In [1]:
import pandas as pd

#Загрузка и предобработка

Основные данные взяты из Кэгла: https://www.kaggle.com/prasertk/netflix-subscription-price-in-different-countries

In [2]:
Netflix_subs = pd.read_csv('https://raw.githubusercontent.com/Andruwkach/pandas_workshop/c735366f40d881a1edccae2d98e1569562db8f63/Netflix.csv')

In [3]:
Netflix_subs.sample(3)

,Country_code,Country,Total Library Size,No. of TV Shows,No. of Movies,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($)
50,sg,Singapore,6303,4109,2194,9.51,12.81,16.11
6,bg,Bulgaria,6797,4819,1978,9.03,11.29,13.54
16,fi,Finland,4045,2638,1407,9.03,13.54,18.06
64,ve,Venezuela,4797,3154,1643,7.99,10.99,13.99
22,hn,Honduras,4989,3154,1835,7.99,10.99,13.99


Данные по душевому доходу по паритету покупательной способности взяты с Кэгла: https://www.kaggle.com/nitishabharathi/gdp-per-capita-all-countries

In [4]:
GDP_PPP = pd.read_csv('https://raw.githubusercontent.com/Andruwkach/pandas_workshop/main/GDP.csv')

In [5]:
GDP_PPP.sample(3)

,Country,Country Code,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
173,Nauru,NRU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3816.382730,5203.830851,5670.057531,6470.453571,7339.304367,8056.696421,10451.939010,13273.483650,13104.480370,13982.201570,15039.317570,16504.257930,NaN
58,Europe & Central Asia (excluding high income),ECA,6524.721226,6336.596376,5791.945909,5553.236198,5034.457335,5028.436261,5079.685668,5267.829853,5312.065554,5481.199632,6121.890196,6483.963187,6950.682153,7663.169600,8588.992459,9658.493778,11553.859110,12895.109650,14815.257360,14330.066870,15296.943900,17372.405280,18296.284230,19017.042700,19482.324480,19251.960790,19592.866130,20906.360670,21931.724450,NaN
8,Antigua and Barbuda,ATG,11326.702620,11806.234410,12005.986790,12682.475350,13524.670290,12924.355370,13729.757550,14416.114870,14951.180660,15423.994600,16526.987220,15804.121560,15994.176150,17059.137160,18284.467660,19784.966790,22634.001000,24994.558220,25064.101770,21852.203080,20134.817250,19876.215320,20672.912060,20656.381580,21599.199140,22422.029390,23670.302260,24644.551880,26868.133520,NaN
45,Curacao,CUW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23271.890210,24497.701940,24953.455940,24969.863150,25256.104130,25558.827310,26084.512840,26910.081020,27683.256880,27566.012800,27559.197050,27907.866020,28168.830180,28115.005800,27941.568190,27947.821670,27679.645030,27626.640950,27743.070050,NaN
101,IDA only,IDX,992.922539,1012.603686,1015.542009,1033.458231,1034.913437,1079.907487,1126.060329,1168.157370,1199.120619,1239.772836,1292.186619,1357.388707,1398.870747,1466.426052,1559.782740,1678.364703,1804.757987,1942.268998,2059.936195,2130.762579,2242.841627,2331.347781,2444.944500,2585.757341,2723.947516,2817.383172,2921.947122,3082.516503,3249.854407,NaN


Возьмём 2017 год как наиболее полный; смёрджим с основными данными; посмотрим на пропуски

In [6]:
GDP_PPP = GDP_PPP[['Country ', '2017']]

In [7]:
Netflix_subs = Netflix_subs.merge(GDP_PPP, how='left', left_on='Country', right_on='Country ')

In [8]:
Netflix_subs[Netflix_subs['2017'].isna()]

,Country_code,Country,Total Library Size,No. of TV Shows,No. of Movies,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($),Country,2017
12,cz,Czechia,7325,5234,2091,8.83,11.49,14.15,NaN,NaN
19,gi,Gibraltar,6167,4079,2088,9.03,14.67,20.32,Gibraltar,NaN
23,hk,Hong Kong,4746,2883,1863,8.08,10.00,11.93,NaN,NaN
33,li,Liechtenstein,3048,1712,1336,12.88,20.46,26.96,Liechtenstein,NaN
38,mc,Monaco,5804,3806,1998,9.03,13.54,18.06,Monaco,NaN
48,ru,Russia,5711,3624,2087,8.13,10.84,13.56,NaN,NaN
51,sk,Slovakia,7035,5055,1980,9.03,11.29,13.54,NaN,NaN
53,kr,South Korea,5195,3334,1861,8.07,11.47,14.45,NaN,NaN
57,tw,Taiwan,5105,3134,1971,9.74,11.90,14.07,NaN,NaN
64,ve,Venezuela,4797,3154,1643,7.99,10.99,13.99,NaN,NaN


Часть данных не смёрджилось, из-за того что в источниках разные названия для одних стран (например, Russia/ Russia Federation), а по части не было данных изначально. Заполним пропуски вручную (взяв недостающие цифры из открытых источников)

In [9]:
Netflix_subs.loc[Netflix_subs.Country == 'Russia', '2017'] = 25766.92881
Netflix_subs.loc[Netflix_subs.Country == 'Czechia', '2017'] = 38019.58051
Netflix_subs.loc[Netflix_subs.Country == 'Gibraltar', '2017'] = 61700
Netflix_subs.loc[Netflix_subs.Country == 'Hong Kong', '2017'] = 61713.41554
Netflix_subs.loc[Netflix_subs.Country == 'Liechtenstein', '2017'] = 157800
Netflix_subs.loc[Netflix_subs.Country == 'Monaco', '2017'] = 116700
Netflix_subs.loc[Netflix_subs.Country == 'Slovakia', '2017'] = 32243.84951
Netflix_subs.loc[Netflix_subs.Country == 'South Korea', '2017'] = 38824.11539
Netflix_subs.loc[Netflix_subs.Country == 'Taiwan', '2017'] = 53200
Netflix_subs.loc[Netflix_subs.Country == 'Venezuela', '2017'] = 18102.46829

Скачаем доп. данные (в частности, по валюте): https://www.comparitech.com/blog/vpn-privacy/countries-netflix-cost/

In [10]:
add_data = pd.read_csv('https://raw.githubusercontent.com/Andruwkach/pandas_workshop/main/additional_data.csv')

In [11]:
add_data.sample(3)

,Country,Currency,Price per Title
15,Mexico,MXN,0.00133
31,Germany,EUR,0.00159
2,Argentina,ARS,0.00079


In [17]:
Netflix_subs = Netflix_subs.merge(add_data, how='left', on='Country')

In [19]:
Netflix_subs[Netflix_subs.Currency.isna()]

,Country_code,Country,Total Library Size,No. of TV Shows,No. of Movies,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($),Country,2017,Currency,Price per Title
12,cz,Czechia,7325,5234,2091,8.83,11.49,14.15,NaN,38019.58051,NaN,NaN


1 пропуск из-за разного названия страны, заполним вручную.

In [20]:
Netflix_subs.loc[Netflix_subs.Country == 'Czechia', 'Currency'] = 'CZK'
Netflix_subs.loc[Netflix_subs.Country == 'Czechia', 'Price per Title'] = 0.00121

Также скачаем данные для определения континента: https://www.kaggle.com/statchaitya/country-to-continent

In [14]:
country_continent = pd.read_csv('https://raw.githubusercontent.com/Andruwkach/pandas_workshop/main/countryContinent.csv', encoding='latin-1')

смотрим, мёрджим, ищем пропуски

In [15]:
country_continent.sample(3)

,country,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
123,Lebanon,LB,LBN,422,ISO 3166-2:LB,Asia,Western Asia,142.0,145.0
181,Romania,RO,ROU,642,ISO 3166-2:RO,Europe,Eastern Europe,150.0,151.0
40,Cabo Verde,CV,CPV,132,ISO 3166-2:CV,Africa,Western Africa,2.0,11.0


In [24]:
country_continent = country_continent[['code_2', 'continent']]
country_continent['code_2'] = country_continent['code_2'].apply(lambda x: str(x).lower())

In [27]:
Netflix_subs = Netflix_subs.merge(country_continent, how='left', left_on='Country_code', right_on='code_2')

In [29]:
Netflix_subs[Netflix_subs.continent.isna()]

,Country_code,Country,Total Library Size,No. of TV Shows,No. of Movies,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($),Country,2017,Currency,Price per Title,code_2,continent


Отлично. За работу!

#Задание

##1 Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от валюты страны.

In [36]:
currency_price = Netflix_subs[
                              [
                                'Cost Per Month - Basic ($)', 
                                'Cost Per Month - Standard ($)', 
                                'Cost Per Month - Premium ($)', 
                                'Currency',
                                ]
                              ].groupby('Currency', as_index=False).mean()

Посмотрим на 5 самых дешёвых подписок по валютам для каждого из видов.

In [38]:
currency_price[
               ['Currency', 'Cost Per Month - Basic ($)']
               ].sort_values('Cost Per Month - Basic ($)').head(5)

,Currency,Cost Per Month - Basic ($)
29,TRY,1.97
15,INR,2.64
0,ARS,3.74
6,COP,4.31
2,BRL,4.61


In [39]:
currency_price[
               ['Currency', 'Cost Per Month - Standard ($)']
               ].sort_values('Cost Per Month - Standard ($)').head(5)

,Currency,Cost Per Month - Standard ($)
29,TRY,3.00
0,ARS,6.30
15,INR,6.61
6,COP,6.86
2,BRL,7.11


In [40]:
currency_price[
               ['Currency', 'Cost Per Month - Premium ($)']
               ].sort_values('Cost Per Month - Premium ($)').head(5)

,Currency,Cost Per Month - Premium ($)
29,TRY,4.02
15,INR,8.60
0,ARS,9.26
6,COP,9.93
2,BRL,9.96


Интересно, что в топ-5 дешёвых подписок входят одни и те же валюты для всех видов подписок: турецкая лира, индийская рупия, аргентинское песо, колумбийское песо, бразильский реал.

Посмотрим на топ-5 дорогих подписок каждого вида.

In [41]:
currency_price[
               ['Currency', 'Cost Per Month - Basic ($)']
               ].sort_values('Cost Per Month - Basic ($)', ascending=False).head(5)

,Currency,Cost Per Month - Basic ($)
4,CHF,12.88
8,DKK,12.00
26,SEK,10.90
14,ILS,10.56
20,NOK,9.94


In [42]:
currency_price[
               ['Currency', 'Cost Per Month - Standard ($)']
               ].sort_values('Cost Per Month - Standard ($)', ascending=False).head(5)

,Currency,Cost Per Month - Standard ($)
4,CHF,20.46
14,ILS,15.05
8,DKK,15.04
26,SEK,14.20
10,GBP,13.20


In [43]:
currency_price[
               ['Currency', 'Cost Per Month - Premium ($)']
               ].sort_values('Cost Per Month - Premium ($)', ascending=False).head(5)

,Currency,Cost Per Month - Premium ($)
4,CHF,26.96
26,SEK,19.70
8,DKK,19.60
14,ILS,19.54
10,GBP,18.48


Топ 5 дорогих подисок покупаются за швейцарский франк, шведская крона, датская крона, новый израильский шекель, фунт стерлингов и, если речь идёт о базовой подписке, норвежскую крону.

##2 Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от континента

In [47]:
continent_price = Netflix_subs[
                              [
                                'Cost Per Month - Basic ($)', 
                                'Cost Per Month - Standard ($)', 
                                'Cost Per Month - Premium ($)', 
                                'continent',
                                ]
                              ].groupby('continent', as_index=False).mean()

In [48]:
continent_price

,continent,Cost Per Month - Basic ($),Cost Per Month - Standard ($),Cost Per Month - Premium ($)
0,Africa,6.260000,10.050000,12.580000
1,Americas,7.223750,10.465625,13.548750
2,Asia,7.636667,10.404167,12.966667
3,Europe,9.230294,13.304412,17.545588
4,Oceania,8.320000,12.325000,16.665000


В Европе самые дорогие цены для любого вида подписки, а второе место всегда занимает Океания. Самые дешёвые подписки в среднем - в Африке.

##3 Сколько стоит подписка относительно среднего дохода (базовая, стандартная, премиум) в Чехии?

Посчитаем, какой % составляет месячная подписка от месячного ВВП на душу населения по ППС

In [55]:
cz_basic_gdp = Netflix_subs[Netflix_subs.Country_code == 'cz'
                ]['Cost Per Month - Basic ($)'
                ] / Netflix_subs[Netflix_subs.Country_code == 'cz'
                ]['2017'] * 12 * 100
cz_standard_gdp = Netflix_subs[Netflix_subs.Country_code == 'cz'
                ]['Cost Per Month - Standard ($)'
                ] / Netflix_subs[Netflix_subs.Country_code == 'cz'
                ]['2017'] * 12 * 100
cz_premium_gdp = Netflix_subs[Netflix_subs.Country_code == 'cz'
                ]['Cost Per Month - Premium ($)'
                ] / Netflix_subs[Netflix_subs.Country_code == 'cz'
                ]['2017'] * 12 * 100

In [60]:
print(f'''базовая подписка = {float(round(cz_basic_gdp, 3))}% от ВВП по ППС на душу,
стандартная подписка = {float(round(cz_standard_gdp, 3))}% от ВВП по ППС на душу,
премиум подписка = {float(round(cz_premium_gdp, 3))}% от ВВП по ППС на душу''')

базовая подписка = 0.279% от ВВП по ППС на душу,
стандартная подписка = 0.363% от ВВП по ППС на душу,
премиум подписка = 0.447% от ВВП по ППС на душу


Каждый чех зарабатывает себе ~ 300 подписок в среднем. Неплохо для людей, которые не пользуются Авито!

##4 Какие топ-5 стран имеют самую большую доступную коллекцию сериалов?

Сериалов в датасете нет, будем надеяться, что автор имел в виду что-то, что уже есть в данных. Например, размер общей библиотеки.

In [67]:
Netflix_subs[
             ['Country', 'Total Library Size']
             ].sort_values('Total Library Size', ascending=False).head(5)

,Country,Total Library Size
12,Czechia,7325
51,Slovakia,7035
24,Hungary,6884
6,Bulgaria,6797
61,United Kingdom,6643


Чехи! И прочие европейцы. Но есть ли у них берёзы, водка и Авито?

##5 Какой процент среднего дохода россиянина будет составлять подписка Netflix?

In [68]:
ru_basic_gdp = Netflix_subs[Netflix_subs.Country_code == 'ru'
                ]['Cost Per Month - Basic ($)'
                ] / Netflix_subs[Netflix_subs.Country_code == 'ru'
                ]['2017'] * 12 * 100
ru_standard_gdp = Netflix_subs[Netflix_subs.Country_code == 'ru'
                ]['Cost Per Month - Standard ($)'
                ] / Netflix_subs[Netflix_subs.Country_code == 'ru'
                ]['2017'] * 12 * 100
ru_premium_gdp = Netflix_subs[Netflix_subs.Country_code == 'ru'
                ]['Cost Per Month - Premium ($)'
                ] / Netflix_subs[Netflix_subs.Country_code == 'ru'
                ]['2017'] * 12 * 100

In [69]:
print(f'''базовая подписка = {float(round(ru_basic_gdp, 3))}% от ВВП по ППС на душу,
стандартная подписка = {float(round(ru_standard_gdp, 3))}% от ВВП по ППС на душу,
премиум подписка = {float(round(ru_premium_gdp, 3))}% от ВВП по ППС на душу''')

базовая подписка = 0.379% от ВВП по ППС на душу,
стандартная подписка = 0.505% от ВВП по ППС на душу,
премиум подписка = 0.632% от ВВП по ППС на душу


Процентов на 30 подороже, чем у Чехов!

##6 На каком месте будет Россия, если отсортировать страны по эффективности подписки (Cost Per Title)

In [75]:
sub_data = Netflix_subs.sort_values('Price per Title'
            ).reset_index()[['Country', 'Price per Title']]

In [76]:
sub_data[sub_data.Country == 'Russia']

,Country,Price per Title
25,Russia,0.00142


26-ое место! Почти в середине

##7 В какой стране лучше всего брать VPN для того, чтобы платить за Netflix меньше всего?

In [78]:
Netflix_subs.sort_values('Cost Per Month - Basic ($)'
            )[['Country', 'Cost Per Month - Basic ($)']].head(1)

,Country,Cost Per Month - Basic ($)
59,Turkey,1.97


In [79]:
Netflix_subs.sort_values('Cost Per Month - Standard ($)'
            )[['Country', 'Cost Per Month - Standard ($)']].head(1)

,Country,Cost Per Month - Standard ($)
59,Turkey,3.0


In [80]:
Netflix_subs.sort_values('Cost Per Month - Premium ($)'
            )[['Country', 'Cost Per Month - Premium ($)']].head(1)

,Country,Cost Per Month - Premium ($)
59,Turkey,4.02


Тур-ци-я! Тур-ци-я! Тур-ци-я!